## Download source code from github

In [ ]:
### Install TensorRT

In [ ]:
!python3 -m pip install --upgrade tensorrt

### Verify that TensorRT installation is working

In [ ]:
import tensorrt
print(tensorrt.__version__)
assert tensorrt.Builder(tensorrt.Logger())

8.6.1


### Test alphageometry

In [ ]:
%%shell

MELIAD_PATH=meliad_lib/meliad
export PYTHONPATH=$PYTHONPATH:/content/alphageometry/$MELIAD_PATH

echo $PYTHONPATH

CUDNN_PATH="/usr/local/lib/python3.10/dist-packages/nvidia/cudnn/"
TENSOR_LIBS="/usr/local/lib/python3.10/dist-packages/tensorrt_libs"
export LD_LIBRARY_PATH=$TENSOR_LIBS:$CUDNN_PATH/lib:$LD_LIBRARY_PATH

echo $LD_LIBRARY_PATH

xvfb-run python alphageometry_test.py

/env/python:/content/alphageometry/meliad_lib/meliad
2024-01-21 21:42:38.626466: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/alphageometry/alphageometry_test.py", line 21, in <module>
    import alphageometry
  File "/content/alphageometry/alphageometry.py", line 28, in <module>
    import lm_inference as lm
  File "/content/alphageometry/lm_inference.py", line 20, in <module>
    import models  # pylint: disable=unused-import
  File "/content/alphageometry/models.py", line 24, in <module>
    from transformer import models
  File "/content/alphageometry/meliad_lib/meliad/transformer/models.py", line 25, in <module>
    import  metrics_summary
  File "/content/alphageometry/meliad_lib/meliad/metrics_summary.py", line 20, in <module>
    from clu import metric_writers
  File "/usr/local/lib/python3.10/dist-packages/clu/metric_writers/__init__.py", line 53, in <module>
    from clu.met

CalledProcessError: Command '
MELIAD_PATH=meliad_lib/meliad
export PYTHONPATH=$PYTHONPATH:/content/alphageometry/$MELIAD_PATH

echo $PYTHONPATH

xvfb-run python alphageometry_test.py
' returned non-zero exit status 1.

In [ ]:
%%shell

MELIAD_PATH=meliad_lib/meliad
export PYTHONPATH=$PYTHONPATH:/content/alphageometry/$MELIAD_PATH

echo $PYTHONPATH

DATA="/content/drive/MyDrive/ag_ckpt_vocab"

echo $DATA

CUDNN_PATH="/usr/local/lib/python3.10/dist-packages/nvidia/cudnn/"
TENSOR_LIBS="/usr/local/lib/python3.10/dist-packages/tensorrt_libs"
export LD_LIBRARY_PATH=$TENSOR_LIBS:$CUDNN_PATH/lib:$LD_LIBRARY_PATH

echo $LD_LIBRARY_PATH

python lm_inference_test.py --meliad_path=$MELIAD_PATH --data_path=$DATA

## Run DDAR

In [ ]:
%%shell

MELIAD_PATH=meliad_lib/meliad
export PYTHONPATH=$PYTHONPATH:/content/alphageometry/$MELIAD_PATH

echo $PYTHONPATH

DATA="/content/drive/MyDrive/ag_ckpt_vocab"

echo $DATA

CUDNN_PATH="/usr/local/lib/python3.10/dist-packages/nvidia/cudnn/"
TENSOR_LIBS="/usr/local/lib/python3.10/dist-packages/tensorrt_libs"
export LD_LIBRARY_PATH=$TENSOR_LIBS:$CUDNN_PATH/lib:$LD_LIBRARY_PATH

echo $LD_LIBRARY_PATH

xvfb-run python -m alphageometry \
--alsologtostderr \
--problems_file=$(pwd)/imo_ag_30.txt \
--problem_name=translated_imo_2000_p1 \
--mode=ddar \
"${DDAR_ARGS[@]}"

## Run AlphaGeometry

In [ ]:
%%shell

MELIAD_PATH=meliad_lib/meliad
export PYTHONPATH=$PYTHONPATH:/content/alphageometry/$MELIAD_PATH

echo $PYTHONPATH

DATA="/content/drive/MyDrive/ag_ckpt_vocab"

echo $DATA

CUDNN_PATH="/usr/local/lib/python3.10/dist-packages/nvidia/cudnn/"
TENSOR_LIBS="/usr/local/lib/python3.10/dist-packages/tensorrt_libs"
export LD_LIBRARY_PATH=$TENSOR_LIBS:$CUDNN_PATH/lib:$LD_LIBRARY_PATH

echo $LD_LIBRARY_PATH

BATCH_SIZE=2
BEAM_SIZE=2
DEPTH=2

SEARCH_ARGS=(
  --beam_size=$BEAM_SIZE
  --search_depth=$DEPTH
)

LM_ARGS=(
  --ckpt_path=$DATA \
  --vocab_path=$DATA/geometry.757.model \
  --gin_search_paths=$MELIAD_PATH/transformer/configs \
  --gin_file=base_htrans.gin \
  --gin_file=size/medium_150M.gin \
  --gin_file=options/positions_t5.gin \
  --gin_file=options/lr_cosine_decay.gin \
  --gin_file=options/seq_1024_nocache.gin \
  --gin_file=geometry_150M_generate.gin \
  --gin_param=DecoderOnlyLanguageModelGenerate.output_token_losses=True \
  --gin_param=TransformerTaskConfig.batch_size=$BATCH_SIZE \
  --gin_param=TransformerTaskConfig.sequence_length=128 \
  --gin_param=Trainer.restore_state_variables=False
);

xvfb-run python -m alphageometry \
--alsologtostderr \
--problems_file=$(pwd)/examples.txt \
--problem_name=orthocenter \
--mode=alphageometry \
"${DDAR_ARGS[@]}" \
"${SEARCH_ARGS[@]}" \
"${LM_ARGS[@]}"